In [ ]:
import os
import json
import shutil
from tqdm import tqdm
import pickle as pk
import pandas as pd
import re
import copy

In [ ]:
adr_fr_R = ""
adr_fr_NR = ""

In [ ]:
def tweetElements(tweet):
    rtFlag = True if "retweeted_status" in tweet.keys() else False
    qtFlag = True if "quoted_status" in tweet.keys() else False 

    if rtFlag == False and qtFlag == False:
        if tweet["truncated"] == True:
            text = copy.deepcopy(tweet["extended_tweet"]["full_text"])
            entities = copy.deepcopy(tweet["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(tweet["text"])
            entities = copy.deepcopy(tweet["entities"])
        user = copy.deepcopy(tweet["user"])
        return [rtFlag, qtFlag, text, entities, user]
    
    elif rtFlag == True and qtFlag == False:
        rt = copy.deepcopy(tweet["retweeted_status"])
        if rt["truncated"] == True:
            text = copy.deepcopy(rt["extended_tweet"]["full_text"])
            entities = copy.deepcopy(rt["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(rt["text"])
            entities = copy.deepcopy(rt["entities"])
        rtUser = copy.deepcopy(rt["user"])
        user = copy.deepcopy(tweet["user"])
        return [rtFlag, qtFlag, text, entities, user, rtUser]
    
    elif rtFlag == False and qtFlag == True:
        if tweet["truncated"] == True:
            text = copy.deepcopy(tweet["extended_tweet"]["full_text"])
            entities = copy.deepcopy(tweet["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(tweet["text"])
            entities = copy.deepcopy(tweet["entities"])
        user = copy.deepcopy(tweet["user"])
        qt = copy.deepcopy(tweet["quoted_status"])
        if qt["truncated"] == True:
            qtText = copy.deepcopy(qt["extended_tweet"]["full_text"])
            qtEntities = copy.deepcopy(qt["extended_tweet"]["entities"])
        else:
            qtText = copy.deepcopy(qt["text"])
            qtEntities = copy.deepcopy(qt["entities"])
        qtUser = copy.deepcopy(qt["user"])
        return [rtFlag, qtFlag, text, entities, user, qtText, qtEntities, qtUser]        
        
    elif rtFlag == True and qtFlag == True:
        rt = copy.deepcopy(tweet["retweeted_status"])
        qt = copy.deepcopy(tweet["quoted_status"])        
        if rt["truncated"] == True:
            text = copy.deepcopy(rt["extended_tweet"]["full_text"])
            entities = copy.deepcopy(rt["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(rt["text"])
            entities = copy.deepcopy(rt["entities"])
        rtUser = copy.deepcopy(rt["user"])        
        if qt["truncated"] == True:
            qtText = copy.deepcopy(qt["extended_tweet"]["full_text"])
            qtEntities = copy.deepcopy(qt["extended_tweet"]["entities"])
        else:
            qtText = copy.deepcopy(qt["text"])
            qtEntities = copy.deepcopy(qt["entities"])
        qtUser = copy.deepcopy(qt["user"])
        user = copy.deepcopy(tweet["user"])
        return [rtFlag, qtFlag, text, entities, user, qtText, qtEntities, qtUser, rtUser]        

In [ ]:
# This loop, iterates over all the collected tweets and copy the ones that meet rumour rules
for file in tqdm(os.listdir(adr_fr_NR)):
    tweet = json.load(open(adr_fr_NR+file))

    elements = tweetElements(tweet)
    rtFlag = copy.deepcopy(elements[0])
    qtFlag = copy.deepcopy(elements[1]) 

    
    if rtFlag == False and qtFlag == False:
        text, entities, user = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4])
    elif rtFlag == True and qtFlag == False:
        text, entities, user, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5])
    elif rtFlag == False and qtFlag == True:
        text, entities, user, qtText, qtEntities, qtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7])       
    elif rtFlag == True and qtFlag == True:
        text, entities, user, qtText, qtEntities, qtUser, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7]), copy.deepcopy(elements[8])
                    
    tweetText = text.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ")
    qtTweetText = qtText.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ") if qtFlag else " "


    flag = False
    if re.search('.*(dishwasher|washing machine).*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*shark.*', tweetText, re.DOTALL) != None :
        flag = True
    elif re.search('.*nuclear.*(danger|leak|burst|collapse|break).*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*(shoot).*(hurricane|storm|florence).*', tweetText, re.DOTALL) != None or re.search('.*fire.*bullet.*(hurricane|storm|florence).*', tweetText, re.DOTALL) != None or \
        re.search('.*(fire|discharge).*(firearm|gun).*(hurricane|storm|florence|air).*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*lootcrew.*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*(dump|throw).*(tampon).*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*(\$10|\$9.8).*(fema|relief fund|recovery fund).*', tweetText, re.DOTALL) != None or \
        re.search('.*(fema|relief fund|recovery fund).*(\$10|\$9.8).*', tweetText, re.DOTALL) != None or \
        re.search('.*(raid|reduce|decrease).*(fema|relief fund|recovery fund).*(budget|fund).*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*(3000|3,000).*die.*puerto(\s|-)?rico.*', tweetText, re.DOTALL) != None or \
        re.search('.*puerto(\s|-)?rico.*death toll.*3000.*', tweetText, re.DOTALL) != None or \
        re.search('.*20.*certified death.*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*(animal|pet).*not.*(hotel|shelter|accomodation).*', tweetText, re.DOTALL) != None or re.search('.*(hotel|shelter|accomodation).*not.*(animal|pet).*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*usaa.*(gas|food|hotel).*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*(rumor|rumour|hoax|fake-?news|fake info|conspirac| myth | myths | scam | scams |disinfo|misinfo|propaganda).*', tweetText, re.DOTALL) != None :
        flag = True
    
    #exception
    if re.search('.*shoot.*looter.*',tweetText, re.DOTALL) != None:
        exceptionFlag = False
    else:
        exceptionFlag = True
    
    qflag = False
    if qtFlag:
        if re.search('.*(dishwasher|washing machine).*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*shark.*', qtTweetText, re.DOTALL) != None :
            qflag = True
        elif re.search('.*nuclear.*(danger|leak|burst|collapse|break).*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*(shoot).*(hurricane|storm|florence).*', qtTweetText, re.DOTALL) != None or re.search('.*fire.*bullet.*(hurricane|storm|florence).*', qtTweetText, re.DOTALL) != None or \
            re.search('.*(fire|discharge).*(firearm|gun).*(hurricane|storm|florence|air).*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*lootcrew.*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*(dump|throw).*(tampon).*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*(\$10|\$9.8).*(fema|relief fund|recovery fund).*', qtTweetText, re.DOTALL) != None or \
            re.search('.*(fema|relief fund|recovery fund).*(\$10|\$9.8).*', qtTweetText, re.DOTALL) != None or \
            re.search('.*(raid|reduce|decrease).*(fema|relief fund|recovery fund).*(budget|fund).*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*(3000|3,000).*die.*puerto(\s|-)?rico.*', qtTweetText, re.DOTALL) != None or \
            re.search('.*puerto(\s|-)?rico.*death toll.*3000.*', qtTweetText, re.DOTALL) != None or \
            re.search('.*20.*certified death.*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*(animal|pet).*not.*(hotel|shelter|accomodation).*', qtTweetText, re.DOTALL) != None or re.search('.*(hotel|shelter|accomodation).*not.*(animal|pet).*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*usaa.*(gas|food|hotel).*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*(rumor|rumour|hoax|fake-?news|fake info|conspirac| myth | myths | scam | scams |disinfo|misinfo|propaganda).*', qtTweetText, re.DOTALL) != None :
            qflag = True 
    if re.search('.*shoot.*looter.*',qtTweetText, re.DOTALL) != None:
        exceptionFlag = False
    else:
        exceptionFlag = True
        
    finalFlag = (flag or qflag) and exceptionFlag
    if finalFlag:
        shutil.move(adr_fr_NR + file, adr_fr_R)